# Fine-tuning

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
# paths to dataset files stored in Google Drive
DATA_DIR = '/content/drive/MyDrive/KeepCoding/Bootcamp_AI/FinalProject/data/processed'

# instructed dataset
PATH_ENVIRONMENT = f'{DATA_DIR}/.env'
PATH_HYROX_SPLIT_DATASET = f'{DATA_DIR}/hyrox_split_dataset'

PATH_HYROX_MODEL_MISTRAL_LORA = f'{DATA_DIR}/hyrox_mistral_lora_model'
PATH_HYROX_MISTRAL_LORA = f'{DATA_DIR}/hyrox_mistral_lora'

PATH_HYROX_OFFLOAD = f'{DATA_DIR}/hyrox_offload'
PATH_HYROX_MODEL_MISTRAL_LORA_2 = f'{DATA_DIR}/hyrox_mistral_lora_2'

In [3]:
!pip install python-dotenv

In [4]:
!pip install fsspec==2025.3.0 gcsfs==2025.3.0 datasets peft transformers accelerate bitsandbytes trl -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.1/367.1 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import os
import pandas as pd
import torch

from datasets import load_from_disk
from dotenv import load_dotenv

from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, BitsAndBytesConfig, pipeline
from peft import PeftModel

In [6]:
from huggingface_hub import notebook_login
notebook_login()

In [7]:
# load the split_hyrox_dataset from disk, saved in the Fine-tuning gpt-2 exercise.
split_hyrox_dataset = load_from_disk('file://' + PATH_HYROX_SPLIT_DATASET)

In [8]:
split_hyrox_dataset['train'][0]

{'input': "Explain this HYROX Open Division participant's performance using their residual and cluster data.",
 'context': 'gender: 1, age_range: 30.0-34.0, total_time: 90, predicted_time: 91, residual: 0, cluster_perf_only: 0, cluster_perf_context: 3',
 'response': 'You performed close to expected levels.  Compared to athletes of similar age and gender, your weak areas were: gender_context, run_1_context, 50m Sled Push_context, roxzone_3_context.',
 'text': "### Instruction:\n    Explain this HYROX Open Division participant's performance using their residual and cluster data.\n\n    ### Context:\n    gender: 1, age_range: 30.0-34.0, total_time: 90, predicted_time: 91, residual: 0, cluster_perf_only: 0, cluster_perf_context: 3\n\n    ### Response:\n    You performed close to expected levels.  Compared to athletes of similar age and gender, your weak areas were: gender_context, run_1_context, 50m Sled Push_context, roxzone_3_context."}

# Load Mistral + Prepare Tokenizer

✅ I will Fine-tune Mistral-7B-Instruct using: Hugging Face transformers, peft for efficient LoRA fine-tuning (GPU-friendly), text column from your split_hyrox_dataset


In [9]:
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",  # best for performance
    bnb_4bit_compute_dtype=torch.float16
)

# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     offload_folder=PATH_HYROX_OFFLOAD,
#     load_in_4bit=False,
#     torch_dtype="auto"
# )

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    offload_folder=PATH_HYROX_OFFLOAD
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

## Prepare Mistral for LoRA Fine-Tuning (with peft)
This step uses Parameter-Efficient Fine-Tuning (LoRA) to only train a small set of weights, which:

	•	Saves memory (works well on Colab GPUs)

	•	Speeds up training
  
	•	Keeps the base Mistral model frozen

In [10]:
# skipped on CPU
model = prepare_model_for_kbit_training(model) # Uncomment this line

In [11]:
# configure LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [12]:
# confirm LoRA is working

model.print_trainable_parameters()

trainable params: 3,407,872 || all params: 7,245,139,968 || trainable%: 0.0470


In [13]:
def tokenize_prompt(ds):
    tokenized = tokenizer(
        ds['text'],
        padding='max_length',
        truncation=True,
        max_length=512,
        return_attention_mask=True,
    )
    tokenized['labels'] = tokenized['input_ids'].copy()
    return tokenized

tokenized_dataset = split_hyrox_dataset.map(tokenize_prompt, batched=True, remove_columns=split_hyrox_dataset["train"].column_names)

Map:   0%|          | 0/133216 [00:00<?, ? examples/s]

Map:   0%|          | 0/16652 [00:00<?, ? examples/s]

Map:   0%|          | 0/16653 [00:00<?, ? examples/s]

In [14]:
# verify include: input_ids, attention_mask
tokenized_dataset['train'][0].keys()
print(tokenized_dataset["train"][0]["attention_mask"])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

## Define TrainingArguments and run training

In [ ]:
# ready to train with HuggingFace Trainer

In [15]:
# first attempt, it took 2 hours for the model to be trained
training_args = TrainingArguments(
    output_dir=PATH_HYROX_MODEL_MISTRAL_LORA,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    num_train_epochs=2,
    learning_rate=1e-4,
    fp16=True,
    logging_steps=25,
    save_strategy="epoch",
    save_total_limit=1,
    report_to="none",
)

In [16]:
small_train = tokenized_dataset['train'].select(range(200))
small_val = tokenized_dataset['val'].select(range(50))

In [17]:
# define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train,     # tokenized_dataset["train"],
    eval_dataset=small_val,        # tokenized_dataset["val"],
    tokenizer=tokenizer
)

/tmp/ipython-input-17-3028575701.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [19]:
# # add 'labels' fields to the tokenized dataset
# def add_labels(ds):
#     ds["labels"] = ds["input_ids"]
#     return ds

# tokenized_dataset = tokenized_dataset.map(add_labels)

Map:   0%|          | 0/110123 [00:00<?, ? examples/s]

Map:   0%|          | 0/13765 [00:00<?, ? examples/s]

Map:   0%|          | 0/13766 [00:00<?, ? examples/s]

- Fine-tune Mistral-7B-Instruct with LoRA on your Hyrox feedback prompts
- Save checkpoints
- Evaluate after each epoch (using your validation set)

In [18]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two vari

Step,Training Loss
25,2.476400


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

TrainOutput(global_step=26, training_loss=2.468626334117009, metrics={'train_runtime': 758.2128, 'train_samples_per_second': 0.528, 'train_steps_per_second': 0.034, 'total_flos': 8741766719078400.0, 'train_loss': 2.468626334117009, 'epoch': 2.0})

In [19]:
model.save_pretrained(PATH_HYROX_MISTRAL_LORA)
tokenizer.save_pretrained(PATH_HYROX_MISTRAL_LORA)

('/content/drive/MyDrive/KeepCoding/Bootcamp_AI/FinalProject/data/processed/hyrox_mistral_lora/tokenizer_config.json',
 '/content/drive/MyDrive/KeepCoding/Bootcamp_AI/FinalProject/data/processed/hyrox_mistral_lora/special_tokens_map.json',
 '/content/drive/MyDrive/KeepCoding/Bootcamp_AI/FinalProject/data/processed/hyrox_mistral_lora/chat_template.jinja',
 '/content/drive/MyDrive/KeepCoding/Bootcamp_AI/FinalProject/data/processed/hyrox_mistral_lora/tokenizer.model',
 '/content/drive/MyDrive/KeepCoding/Bootcamp_AI/FinalProject/data/processed/hyrox_mistral_lora/added_tokens.json',
 '/content/drive/MyDrive/KeepCoding/Bootcamp_AI/FinalProject/data/processed/hyrox_mistral_lora/tokenizer.json')

In [20]:
trainer.push_to_hub()

training_args.bin:   0%|          | 0.00/5.50k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Syllerim/hyrox_mistral_lora_model/commit/60f2e5177b0df4a3aa756221e1942f23a6c48d80', commit_message='End of training', commit_description='', oid='60f2e5177b0df4a3aa756221e1942f23a6c48d80', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Syllerim/hyrox_mistral_lora_model', endpoint='https://huggingface.co', repo_type='model', repo_id='Syllerim/hyrox_mistral_lora_model'), pr_revision=None, pr_num=None)

In [35]:
# # base model ID and LoRA adapter path
# base_model_id = "mistralai/Mistral-7B-Instruct-v0.1"
# lora_adapter_path = PATH_HYROX_MISTRAL_LORA

# # load tokenizer
# tokenizer = AutoTokenizer.from_pretrained(base_model_id)

# # load base model
# base_model = AutoModelForCausalLM.from_pretrained(
#     base_model_id,
#     device_map="auto",
#     torch_dtype="auto"
# )

# # apply the LoRA adapter
# base_model = AutoModelForCausalLM.from_pretrained(base_model_id, device_map="auto", torch_dtype="auto")
# model = PeftModel.from_pretrained(base_model, lora_adapter_path)
# model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for base_model.model.model.layers.8.self_attn.q_proj.lora_A.default.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for base_model.model.model.layers.8.self_attn.q_proj.lora_B.default.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for base_model.model.model.l

KeyError: 'base_model.model.model.model.embed_tokens'

In [23]:
!accelerate config

--------------------------------------------------------------------------------In which compute environment are you running?
Please input a choice index (starting from 0), and press enter
 ➔  This machine
    AWS (Amazon SageMaker)
0
This machine
--------------------------------------------------------------------------------Which type of machine are you using?
Please input a choice index (starting from 0), and press enter
 ➔  No distributed training
    multi-CPU
    multi-XPU
    multi-HPU
    multi-GPU
    multi-NPU
    multi-MLU
    multi-SDAA
    multi-MUSA
    TPU
0
No distributed training
Do you want to run your training on CPU only (even if a GPU / Apple Silicon / Ascend NPU device is available)? [yes/NO]:NO
Do you wish to optimize your script with torch dynamo?[yes/NO]:NO
Do you want to use DeepSpeed? [yes/NO]: NO
What GPU(s) (by id) should be used for training on this machine as a comma-separated list? [all]:all
Would you like to enable numa efficiency? (Currently only suppo

In [24]:
import torch

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    PATH_HYROX_MISTRAL_LORA,
    # device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    # offload_folder=PATH_HYROX_OFFLOAD
)

tokenizer = AutoTokenizer.from_pretrained(PATH_HYROX_MISTRAL_LORA)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [31]:
from accelerate import dispatch_model

In [ ]:
model = dispatch_model(
    model,
    device_map="auto",
    offload_dir="./offload"  # 👈 este sí lo acepta
)

In [26]:
# Use the generation pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

# Prompt
prompt = """Generate performance feedback based on prediction and true time
gender: 1, age_range: 30-34, total_time: 3800, predicted_time: 3600, residual: 200, cluster_perf_only: 2, cluster_perf_context: 1
### Response:"""

# Generate response
outputs = generator(prompt, max_new_tokens=150, do_sample=False)
print(outputs[0]["generated_text"])

ValueError: The model has been loaded with `accelerate` and therefore cannot be moved to a specific device. Please discard the `device` argument when creating your pipeline object.

In [28]:
# # load the fine-tuned checkpoint
# model_path = PATH_HYROX_MISTRAL_LORA

# tokenizer = AutoTokenizer.from_pretrained(model_path)
# model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")

# # use the generation pipeline
# generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# prompt = """Generate performance feedback based on prediction and true time
# gender: 1, age_range: 30-34, total_time: 3800, predicted_time: 3600, residual: 200, cluster_perf_only: 2, cluster_perf_context: 1
# ### Response:"""

# outputs = generator(prompt, max_new_tokens=150, do_sample=False)
# print(outputs[0]['generated_text'])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for model.layers.10.self_attn.q_proj.lora_A.default.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for model.layers.10.self_attn.q_proj.lora_B.default.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for model.layers.10.self_attn.v_proj.lora_A.default.weight: 

ValueError: We need an `offload_dir` to dispatch this model according to this `device_map`, the following submodules need to be offloaded: model.norm, model.rotary_emb, lm_head, model.layers.